Dans ce tutoriel, nous allons explorer certaines méthodes de traitement de signal qui pourraient vous être utiles. Ce n'est **pas** un primer sur le traitement de signal, puisque je ne suis pas la personne la mieux placée pour cela.

Toutefois, après avoir fait de la recherche sur le sujet, j'ai choisi trois outils qui pourraient vous être utiles. En effet, les deux premiers outils servent à diminuer la quantité de bruit dans un signal, tandis que le dernier permet de comparer des signaux.

**Petit disclaimer** Je ne suis pas expert en traitement de signal. J'ai fait de la recherche et j'ai consulté les notes du cours [IFT-7030](https://ycemsubakan.github.io/mlsp.html) avant de faire un choix sur les méthodes que je considérais les plus importantes.

De plus, j'ai masqué beaucoup de code dans Colab puisqu'il est uniquement nécessaire pour faire des beaux graphiques. Vous pouvez regarder le code, mais ce n'est pas le but du tutoriel.

L'importation des librairies est relativement lent, donc exécutez tout de suite cette cellule. J'ai caché les sorties, si jamais vous avez un problème, enlevez `%%capture` puis regardez le message d'erreur.

In [ ]:
# @title Importations des librairies
%%capture
!pip install dtaidistance
!pip install PyWavelets
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
from dtaidistance.subsequence.dtw import subsequence_alignment
import numpy as np
import random
import matplotlib.pyplot as plt
import pywt
import torch
from scipy.ndimage import gaussian_filter1d, uniform_filter1d
from scipy.signal import savgol_filter

# Transformée de Fourier

Le premier outil que je vous présente est la transformée de Fourier, spécifiquement la Discrete Fourier Transform. Si vous ne connaissez pas, je vous recommande le vidéo suivant sur le sujet : [The Discrete Fourier Transform: Most Important Algorithm Ever?](https://youtu.be/yYEMxqreA10?si=_xdnYMWyjT6h48BG)


Dans le cadre du traitement de signal, la transformée de Fourier est extrêmement importante, entre autres pour l'utilisation que je vais présenter aujourd'hui.

La transformée de Fourier nous permet de représenter le signal obtenu comme une somme de sinus et de cosinus. En ne gardant que les termes ayant les coefficients les plus grands, on peut obtenir une version plus lisse de notre signal.

Pour démontrer ces faits, on va commencer par faire un exemple avec un signal non-bruité. J'ai caché le code pour ne pas que vous sachiez quels sont les termes que j'ai additionnés pour obtenir ce signal. Essayez de deviner, c'est une somme de sinus et de cosinus.


Source : L'utilisation de la fonction *fft* est inspirée de [ce blog](https://www.artofai.dev/posts/fft/). La matrice de sinus et cosinus est inspirée des notes de cours de [IFT-7030](https://ycemsubakan.github.io/mlsp.html).

In [ ]:
# @title
sampling_rate = 1000
duration = 2

x = np.linspace(0, duration, duration * sampling_rate)

signal = np.sin(2*np.pi*6*x) + 4*np.sin(2*np.pi*x) + np.cos(2*np.pi* 4 * x) + (1/5)*np.cos(2*np.pi*20*x)

plt.plot(x, signal)
plt.xlabel("x")
plt.show()

Une première approche qu'on pourrait essayer serait d'utiliser le produit scalaire (*dot product*) entre une matrice de sinus et de cosinus et notre signal. On sait que le produit scalaire est une bonne mesure de similarité, donc ça pourrait nous donner une bonne idée. Plus les signaux sont proches, plus le produit scalaire va être grand.

Construisons maintenant une matrice qui contient des sinus et des cosinus. Pour des raisons de visualisations, les signaux sont dispersés sur l'axe des y.

In [ ]:
# @title
nbr_of_parameters = 30
sines = np.zeros((2*nbr_of_parameters, duration*sampling_rate))

for i in range(nbr_of_parameters):
  sines[i, :] = np.sin(2*np.pi * (i+1)* x)
  sines[i+nbr_of_parameters, : ] = np.cos(2*np.pi * (i+1)* x)
  if i == 0:
    plt.plot(x, sines[i, :]+2*i, color="red", label="sinus")
    plt.plot(x, sines[i+nbr_of_parameters, :]+ 2*(i + nbr_of_parameters), color="blue", label="cosinus")
  else:
    plt.plot(x, sines[i, :]+2*i, color="red")
    plt.plot(x, sines[i+nbr_of_parameters, :]+ 2*(i + nbr_of_parameters), color="blue")

plt.xlabel("Duration")
plt.legend()
plt.show()

On calcule maintenant la similarité entre les signaux de références et notre signal. On imprime seulement les termes qui sont les plus similaires. Retenez bien les termes et valeurs obtenues.

In [ ]:
# @title
res = sines @ signal
indexs = res.argsort()[::-1]

freqs = np.arange(nbr_of_parameters) + 1
for i in indexs[:4]:
  if i > nbr_of_parameters:
    print(f"terme = cos({freqs[i % nbr_of_parameters]}x), valeur={res[i]/sampling_rate:.4f}")
  else:
    print(f"terme = sin({freqs[i % nbr_of_parameters]}x), valeur={res[i]/sampling_rate:.4f}")

Nous allons maintenant utiliser la transformée de Fourier fournie par PyTorch. Cette méthode est beaucoup plus rapide que l'approche naïve que nous avons implémentées, et permet d'éviter de stocker en mémoire des dizaines ou des centaines de vecteurs, qui sont tous potentiellement extrêmement longs.

La transformée discrète de Fourier (implémentée par `torch.fft.fft`) transforme une vecteur de N nombre complexes $\mathbf{x} = [x_1, x_2, \ldots, x_N]$ en une séquence de nombre complexes $\mathbf{F} = [f_1, f_2, \ldots, f_N]$ grâce à la fonction suivante :

$$f_k = \sum_{n=1}^{N}x_n e^{-i 2 \pi \frac{k}{N} n}$$

Avec la formule d'Euler $e^{i \omega t} = \cos(\omega t) + i \sin(\omega t)$, on peut réécrire chaque coefficient de la façon suivante:

$$f_k = \sum_{n=1}^{N}x_n e^{-i 2 \pi \frac{k}{N} n}=  \sum_{n=1}^{N}x_n \cos\big(- 2 \pi \frac{k}{N} n\big) + i \sum_{n=1}^{N}x_n\sin\big(-2 \pi \frac{k}{N} n\big).$$

Chaque terme peut se réécrire comme le produit scalaire entre $\mathbf{x}$ et un sinus/cosinus de fréquence $\omega = \frac{k}{N}$. On projete donc exactement en même temps sur les sinus et les cosinus, similairement à ce que l'on a fait dans la cellule précédente.

Source : [Discrete Fourier Transform](https://en.wikipedia.org/wiki/Discrete_Fourier_transform)

Observez maintenant le graphique ainsi que les termes obtenus. Ce graphique correspond à la magnitude des coefficients de fourier pour chaque fréquence considérée.

In [ ]:
transform = torch.fft.fft(torch.tensor(signal))

magnitude = transform.abs()[:len(signal)//2] # abs n'est pas la valeur absolue, mais la magnitude d'un nombre complexe

freq = torch.fft.fftfreq(len(signal), 1/sampling_rate) # on va chercher les fréquences considérées par fft.

plt.plot(freq[:50], magnitude[:50])
plt.xlabel("Fréquence")
plt.ylabel("Magnitude")
plt.show()
sorted_mags = magnitude.argsort(descending=True)
for i in sorted_mags[:4]:
    print(f'Fréquence={freq[i]:.2f}\t Magnitude={magnitude[i]/sampling_rate:.4f}')

Nous pouvons maintenant utiliser ces termes pour reconstruire notre signal. Reconstruire le signal ne sera probablement pas nécessaire pour vous, mais il existe une fonction de PyTorch qui s'appelle `torch.fft.ifft`, que nous utiliserons plus tard.

On voit bien que la représentation obtenue grâce à la transformée de Fourier est exactement le signal original utilisé. La transformée de Fourier nous a donc permis de retrouver les fréquences qui se trouvaient dans le signal sinusoïdale ainsi que la magnitude de chacun des signaux.

In [ ]:
# @title
recon_signal = np.zeros(signal.shape)

print("signal = ", end="")
for i in sorted_mags[:4]:
  mag = (magnitude[i]/sampling_rate).item()
  if transform[i].real.abs() < transform[i].imag.abs():
    recon_signal = mag * np.sin(2*np.pi * freq[i] * x) + recon_signal
    print(f"{mag:.2f}*sin({freq[i]:.2f}x)", end=("+ " if i != sorted_mags[3] else ""))
  else:
    recon_signal = mag * np.cos(2*np.pi * freq[i] * x) + recon_signal
    print(f"{mag:.2f}*cos({freq[i]:.2f}x)", end=("+ " if i != sorted_mags[3] else ""))

fig, axs = plt.subplots(nrows=1,ncols=2, figsize=(10,5))
axs[0].plot(x, recon_signal)
axs[0].set_title("Signal reconstruit")
axs[1].plot(x, signal)
axs[1].set_title("Signal original")
plt.show()

Retournez maintenant voir la première cellule où l'on construit le message, pour comparer les termes qu'on a trouvés avec les vrais termes.

## Exemple bruité

Nous allons maintenant utiliser la transformée de Fourier sur un signal non-sinusoïdale. On va essayer de représenter le signal avec des sinus et des cosinus.

Commençons avec un signal aléatoire.

In [ ]:
# @title
# code de la marche aléatoire inspirée de https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter1d.html#gaussian-filter1d
sampling_rate = 1000
duration = 2

x = np.linspace(0, duration, duration * sampling_rate)

rng = np.random.default_rng(14)
signal = rng.standard_normal(duration*sampling_rate).cumsum()

plt.plot(x, signal)
plt.xlabel("x")
plt.show()

Calculons maintenant la transformée de Fourier. On voit bien qu'il n'existe pas un sous-ensemble clair de fréquences qui représentent le signal.

In [ ]:
transform = torch.fft.fft(torch.tensor(signal))

magnitude = transform.abs()

freq = torch.fft.fftfreq(len(signal), 1/sampling_rate)

plt.plot(freq[:len(signal)//2], magnitude[:len(signal)//2])
plt.xlabel("Fréquence")
plt.ylabel("Magnitude")
plt.show()
sorted_mags = magnitude[:len(signal)//2].argsort(descending=True)
for i in sorted_mags[:5]:
    print(f'Fréquence={freq[i]:.2f}\t Magnitude={magnitude[i]/sampling_rate:.4f}')

On va maintenant reconstruire le signal en utilisant la fonction `torch.fft.ifft`. Grâce au slider, choisissez le nombre de termes qui seront utilisés pour reconstruire le signal.

In [ ]:
# @title {run:"auto"}
nombre_de_termes = 12 # @param {"type":"slider","min":2,"max":40,"step":2}

sorted_mag_both = magnitude[:len(signal)//2].argsort(descending=True)
recon_transform = torch.zeros(transform.shape, dtype=transform.dtype)
recon_transform[sorted_mag_both[:nombre_de_termes]] = transform[sorted_mag_both[:nombre_de_termes]]

fig, axs = plt.subplots(nrows=1,ncols=3, figsize=(15,5))

axs[0].plot(x, torch.fft.ifft(recon_transform).real)
axs[0].set_title(f"Reconstructed function with {nombre_de_termes} terms")
axs[1].plot(x, torch.fft.ifft(transform).real)
axs[1].set_title("Reconstructed function with full transform")
axs[2].plot(x, signal)
axs[2].set_title("Original transform")

plt.show()

Lorsque le nombre de terme est assez grand, on voit que l'on est capable d'obtenir une bonne approximation du signal.

# Smoothing of signal

Parfois, un signal contient trop de bruit et on aimerait le lisser, pour avoir des meilleurs résultats. On a montré qu'on pouvait utiliser la transformée de Fourier pour simplifier un signal, mais parfois on veut simplement lisser le signal.

En lissant le signal, on va obtenir un nouveau signal qui suit la tendance du signal original mais qui évite certains pics liés au bruit.

*Source* : Inspiré de la [documentation de scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter1d.html#gaussian-filter1d)

Commençons donc par choisir un signal construit de façon aléatoire.

In [ ]:
# @title
rng = np.random.default_rng(14)
x = rng.standard_normal(101).cumsum()

plt.figure(figsize=(10,7))
plt.plot(x, 'k', label='Noisy data')
plt.legend()
plt.grid()
plt.show()

Pour lisser un signal, on effectue une [convolution](https://en.wikipedia.org/wiki/Convolution) avec un filtre.

Par exemple, supposons qu'on prend un filtre uniforme de taille 3. On a donc un filtre $[\frac{1}{3}, \frac{1}{3}, \frac{1}{3}]$. Prenons maintenant un signal $\mathbf{x} = [x_1, \ldots, x_N]$.

Le signal lissé $\mathbf{y} = [y_1, \ldots, y_N]$ est donc défini de la façon suivante :
$$y_n = \frac{1}{3}x_{n-1}+ \frac{1}{3}x_{n} +  \frac{1}{3}x_{n+1} $$.

Si on utilise un filtre plus large, chaque valeur du signal lissé va être la moyenne de beaucoup plus de données. Plus on ajoute des données, plus le signal est lisse.

Essayons maintenant le filtre uniforme de scipy.

In [ ]:
y3 = uniform_filter1d(x, 3)
y5 = uniform_filter1d(x, 5)
y15 = uniform_filter1d(x, 15)

plt.figure(figsize=(10,7))
plt.plot(x, 'k', label='original data')
plt.plot(y3, '--', label='filtered, length=3')
plt.plot(y5, ':', label='filtered, length=5')
plt.plot(y15, ':', label='filtered, length=15')
plt.legend()
plt.grid()
plt.show()

Un autre filtre très commun est le filtre gaussien $$g(x) = \exp\big(-\frac{x^2}{2 \sigma^2}\big).$$

On effectue encore une fois une convolution, mais cette fois-ci le noyau est gaussien au lieu d'être uniforme. Cela donne moins de poids aux données qui sont plus loin du centre.


In [ ]:
y3 = gaussian_filter1d(x, 3, truncate=4) # smoothes on 25 datapoints
y6 = gaussian_filter1d(x, 6, truncate=4) # smoothes on 49 datapoints
y30 = gaussian_filter1d(x, 10, truncate=4) # smoothes on 81 datapoints

plt.figure(figsize=(10,7))
plt.plot(x, 'k', label='original data')
plt.plot(y3, '--', label='filtered, sigma=3')
plt.plot(y6, ':', label='filtered, sigma=6')
plt.plot(y30, ':', label='filtered, sigma=10')
plt.legend()
plt.grid()
plt.show()

Finalement, j'ai mis un filtre que j'ai vu recommandé plusieurs fois en faisant mes recherches. C'est le filtre Savitzky-Golay.

In [ ]:
y10_1 = savgol_filter(x, 10, 1)
y20_3 = savgol_filter(x, 20, 3)

plt.figure(figsize=(10,7))
plt.plot(x, 'k', label='original data')
plt.plot(y10_1, '--', label='filtered, length=10, order=1')
plt.plot(y20_3, ':', label='filtered, length=20, order=3')
plt.legend()
plt.grid()
plt.show()

Il n'y a pas de consensus sur le meilleur filtre, vous devrez les essayer pour voir lequel fonctionne le mieux pour vous. Toutefois, mon intuition me dit que le filtre gaussien devrait toujours bien fonctionner.

# Dynamic Time Warping

Supposons que l'on veut comparer deux signaux. Même s'ils sont extrêmement similaires, s'ils sont légèrement décalés dans le temps, la comparaison avec le produit scalaire va nous dire que les signaux sont complètement différents.

Ainsi, parfois, on voudrait être capables de comparer des signaux qui sont possiblement décalés. C'est à cela que va servir le [Dynamic Time Warping](https://en.wikipedia.org/wiki/Dynamic_time_warping).

Le DTW est un problème de programmation dynamique. On va donc essayer d'aligner les deux signaux en calculant toutes les possibilités. Supposons qu'on a deux signaux $f(t)$ et $g(t)$. Alors, on va comparer $f(1)$ à $g(1)$, puis $f(1)$ à $g(2)$, jusqu'à $g(N)$. Puis on va recommencer avec $f(2)$, puis $f(3)$, jusqu'à $f(N)$.

On a donc construit une matrice de distance entre $f(t)$ et $g(t)$ et on va choisir le chemin dans la matrice qui minimise la distance entre les deux signaux.


Source :  
Les exemples sont inspirés de la documentation de [dtaidistance](https://dtaidistance.readthedocs.io/en/latest/index.html)

L'explication est basée sur les slides du cours [IFT-7030](https://ycemsubakan.github.io/class_slides/mlsp_week10.pdf).

Regardons maintenant un exemple où l'on va comparer deux sinus déphasés et légèrement bruités. La ligne rouge nous indique le chemin optimal.

In [ ]:
rng = np.random.default_rng(14)
x = np.arange(0, 20, .5)
s1 = np.sin(x)
s2 = np.sin(x - 2)
random.seed(1)
for idx in range(len(s2)):
  s2[idx] += (rng.uniform() - 0.5) / 5
d, paths = dtw.warping_paths(s1, s2, window=25, psi=2)
best_path = dtw.best_path(paths)
dtwvis.plot_warpingpaths(s1, s2, paths, best_path)
plt.show()

En effet, regardons maintenant la comparaison entre les points des signaux. On voit qu'effectivement, les points sélectionnés se trouvent à la même hauteur sur les sinus. On a donc réussi à aligner les deux sinus déphasés.

In [ ]:
# @title
fig = plt.figure(figsize=(10,10))
fig, axs = dtwvis.plot_warpingpaths(s1, s2, paths, best_path, figure=fig)
axs[3].axhline(y=4, xmin=0, xmax=0.21, color = "orange")
axs[2].axhline(y=36, color="orange")
axs[3].axvline(x=8,ymin=0.89, ymax=1, color="orange")
axs[1].axvline(x=8, color="orange")

axs[3].axhline(y=9, xmin=0, xmax=0.335, color = "orange")
axs[2].axhline(y=40-9, color="orange")
axs[3].axvline(x=13,ymin=0.765, ymax=1, color="orange")
axs[1].axvline(x=13, color="orange")

axs[3].axhline(y=16, xmin=0, xmax=0.51, color = "orange")
axs[2].axhline(y=40-16, color="orange")
axs[3].axvline(x=20,ymin=0.59, ymax=1, color="orange")
axs[1].axvline(x=20, color="orange")

axs[3].axhline(y=22, xmin=0, xmax=0.66, color = "orange")
axs[2].axhline(y=40-22, color="orange")
axs[3].axvline(x=26,ymin=0.44, ymax=1, color="orange")
axs[1].axvline(x=26, color="orange")

axs[3].axhline(y=28, xmin=0, xmax=0.81, color = "orange")
axs[2].axhline(y=40-28, color="orange")
axs[3].axvline(x=32,ymin=0.29, ymax=1, color="orange")
axs[1].axvline(x=32, color="orange")

axs[3].axhline(y=34, xmin=0, xmax=0.96, color = "orange")
axs[2].axhline(y=40-34, color="orange")
axs[3].axvline(x=38,ymin=0.14, ymax=1, color="orange")
axs[1].axvline(x=38, color="orange")

plt.show()

Finalement, supposons qu'on a une série de signaux ainsi qu'une requête (*query*). On voudrait essayer de matcher la requête avec une section, ou plusieurs sections, du signal.

Dans le cas de cette série spécifiquement, c'est une suite de deux ondelettes qui sont multipliées par un nombre aléatoire, les rendant plus ou moins étirées sur l'axe des y.

In [ ]:
# @title
rng = np.random.default_rng(14)
[phi, psi, x] = pywt.Wavelet('db2').wavefun(level=4)

large_x = []
y = []
for i in range(10):
  large_x.append(3*i+x)
  val = 1 + 0.8 * rng.standard_normal(1)
  if i % 2 == 0:
    y.append(val*psi)
  else:
    y.append(val*phi)

large_x = np.concatenate(large_x)
y = np.concatenate(y)

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10,10))
axs[0].plot(large_x, y)
axs[0].set_title("Serie")
axs[1].plot(x, psi)
axs[1].set_title("Query")

query = psi
series = y

Alors, on peut utiliser l'alignement de sous-séquence de la librairie [dtaidistance](https://dtaidistance.readthedocs.io/en/latest/index.html) pour trouver la sous-séquence de la série qui matche le mieux avec notre requête. On observe qu'effectivement, la sous-séquence trouvée semble matcher plutôt bien avec notre requête.

In [ ]:
fig = plt.figure(figsize=(15,15))
sa = subsequence_alignment(query, series)
match_ = sa.best_match()
startidx, endidx = match_.segment
dtwvis.plot_warpingpaths(query, series, sa.warping_paths(), match_.path, figure=fig)

Toutefois, il semble qu'il y ait d'autres sous-séquences qui auraient pu être un bon match. Dans ce cas, il est possible d'avoir plusieurs sous-séquences potentielles en utilisant la fonction `kbest_matches` au lieu de `best_match`.